In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist # 讀入 Fashion MNSIT 數據集

## 1. 資料前處理

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [4]:
print(f'訓練資料總筆數為 {len(x_train)} 筆資料',f'測試資料總筆數為 {len(x_test)} 筆資料',sep='\n')

訓練資料總筆數為 60000 筆資料
測試資料總筆數為 10000 筆資料


In [23]:
classes_dict = {
    0:'T-shirt/top',
    1:'Trouser',
    2:'Pullover',
    3:'Dress',
    4:'Coat',
    5:'Sandal',
    6:'Shirt',
    7:'Sneaker',
    8:'Bag',
    9:'Ankle boot'
}

In [6]:
print(x_train.shape, x_test.shape, sep="\n") # 查看數據型態

(60000, 28, 28)
(10000, 28, 28)


In [7]:
# 1 stands for one channel(greyscale)
# normalization [0,1]
x_train = x_train.reshape(60000, 28, 28, 1) / 255 
x_test = x_test.reshape(10000, 28, 28, 1) /255

In [8]:
from tensorflow.keras.utils import to_categorical

In [9]:
# one-hot encoding
y_train = to_categorical(y_train, 10) #[0,1,2,3,4,5,6,7,8,9]
y_test = to_categorical(y_test, 10)

### 2. 打造CNN函數學習機

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam

In [11]:
model = Sequential() # 打開空白函數學習機

In [12]:
# 卷積(filters/kernals, filter_dim, Zero Padding, activation func)
model.add(Conv2D(16, (3,3), padding = 'same', input_shape = (28,28,1), activation = 'relu'))

# 池化，減少參數以避免overfitting
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(32,(3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(64,(3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

# 拉平(全連結網路無法讀取矩陣資料)
model.add(Flatten())

# fully connected 全連結神經網路
model.add(Dense(100,activation='relu'))
model.add(Dense(60,activation='relu'))

# (輸出10個數字，加起來和為1)
model.add(Dense(10, activation='softmax')) 

In [13]:
model.summary() #檢查神經網路

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

In [16]:
#組裝神經網路
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

## 3. fit(訓練)

In [17]:
model.fit(x_train, y_train, batch_size=100, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 32s 529us/sample - loss: 0.0302 - accuracy: 0.7819
Epoch 2/10
60000/60000 [==============================] - 32s 525us/sample - loss: 0.0184 - accuracy: 0.8731
Epoch 3/10
60000/60000 [==============================] - 33s 546us/sample - loss: 0.0159 - accuracy: 0.8900
Epoch 4/10
60000/60000 [==============================] - 34s 572us/sample - loss: 0.0144 - accuracy: 0.9021
Epoch 5/10
60000/60000 [==============================] - 37s 612us/sample - loss: 0.0132 - accuracy: 0.9103
Epoch 6/10
60000/60000 [==============================] - 36s 592us/sample - loss: 0.0123 - accuracy: 0.9167
Epoch 7/10
60000/60000 [==============================] - 34s 567us/sample - loss: 0.0116 - accuracy: 0.9214
Epoch 8/10
60000/60000 [==============================] - 35s 578us/sample - loss: 0.0109 - accuracy: 0.9262
Epoch 9/10
60000/60000 [==============================] - 35s 579us/sample - loss: 0.0105 - accuracy: 0.9

## 4. 預測

In [20]:
from ipywidgets import interact_manual

In [18]:
result = model.predict_classes(x_test)

In [24]:
def my_predict(n):
    print('我的 CNN 預測是', classes_dict[result[n]])
    X = x_test[n].reshape(28,28)
    plt.imshow(x_test[n].reshape(28,28),cmap='Greys')
interact_manual(my_predict,n=(0,9999))

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.my_predict>

In [29]:
score = model.evaluate(x_test,y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [30]:
loss, acc = score
print("測試資料正確率為", acc)

測試資料正確率為 0.9083


# 改變的地方
* Dense layers = 3
* Optimizer = Adam
* batch size = 100
* epochs = 10